In [20]:
import pandas as pd
import numpy as np

In [21]:
df = pd.read_csv('data/2015_curr.csv')

In [22]:
import plotly.express as ex

In [23]:
ex.line(y=df['Close'])

In [24]:
close = df['Close']
high = df['High']
low = df['Low']
open = df['Open']
volume = df['Volume']

In [25]:
r_t = np.log(close / close.shift(1))

In [26]:
ex.line(y=r_t)

In [27]:
sma_ranges = [5, 10, 20]
ema_ranges = [5, 10]

# 1. Create a DataFrame from your raw data
close_df = pd.DataFrame(close)

for r in sma_ranges:
    close_df[f'SMA {r}'] = close_df['Close'].rolling(r).mean()

for r in ema_ranges:
    close_df[f'EMA {r}'] = close_df['Close'].ewm(span=r, adjust=False).mean()


fig = ex.line(
    close_df,
    title='Stock Price and Moving Averages (Plotly Express)',
    labels={'index': 'Index', 'value': 'Price', 'variable': 'Legend'} # Renames labels
)

# 4. Show the plot
fig.show()

In [28]:
ema_12 = close.ewm(span=12, adjust=False).mean()
ema_26 = close.ewm(span=26, adjust=False).mean()
close_df['MACD'] = ema_12 - ema_26
close_df['MACD Signal'] = close_df['MACD'].ewm(span=9, adjust=False).mean()

ex.line(close_df)

In [29]:
delta = close.diff()
up = delta.clip(lower=0)
down = -1 * delta.clip(upper=0)
avg_up = up.ewm(alpha=1/14, adjust=False).mean()
avg_down = down.ewm(alpha=1/14, adjust=False).mean()
rs = avg_up / (avg_down)
close_df['rsi_14'] = 100 - (100 / (1 + rs))

ex.line(close_df['rsi_14'])

In [30]:
bb_window = 20
bb_kstd = 2

close_df['Middle Band'] = close.rolling(window=bb_window).mean()
std = close.rolling(window=bb_window).std()

close_df['Upper Band'] = close_df['Middle Band'] + (bb_kstd * std)
close_df['Lower Band'] = close_df['Middle Band'] - (bb_kstd * std)

In [31]:
import plotly.graph_objects as go

fig = go.Figure()

fig.add_trace(go.Scatter(
    x=close_df.index,
    y=close_df['Lower Band'],
    name='Lower Band',
    line=dict(color='lightgray', width=1, dash='dash'),
))

fig.add_trace(go.Scatter(
    x=close_df.index,
    y=close_df['Upper Band'],
    name='Upper Band',
    line=dict(color='lightgray', width=1, dash='dash'),
    fill='tonexty', 
    fillcolor='rgba(200, 200, 200, 0.2)'
))

fig.add_trace(go.Scatter(
    x=close_df.index,
    y=close_df['Close'],
    name='Close Price',
    line=dict(color='blue', width=2)
))

fig.add_trace(go.Scatter(
    x=close_df.index,
    y=close_df['Middle Band'],
    name=f'SMA {bb_window} (Middle Band)',
    line=dict(color='orange', width=1, dash='dash')
))

# --- 3. Update Layout & Show ---

fig.update_layout(
    title=f'Bollinger Bands ({bb_window}, {bb_kstd})',
    xaxis_title='Period',
    yaxis_title='Price',
    legend_title='Indicator',
    hovermode='x unified'
)

fig.show()

In [32]:
ex.line(df['Volume'].ewm(span=10, adjust=False).mean())

In [33]:
df['obv'] = (np.sign(df['Close'].diff()).fillna(0) * df['Volume']).cumsum()


In [34]:
ex.line(df['obv'].diff().rolling(5).mean())

In [36]:
df2 = pd.read_csv('features.csv', index_col=0)
df2.head()

,macd,macd_signal,rsi_14,bb_width,close_div_sma20,zscore_ret_20,atr_pct,logret_lag_1,logret_lag_2,logret_lag_3,logret_lag_5,logret_lag_8,logret_lag_13,ret_mean_5,ret_mean_10,ret_mean_20,ret_std_5,ret_std_10,ret_std_20,target_cls
49,-0.513927,-3.370606,52.117930,0.181110,1.047016,0.320433,0.065153,-0.030356,0.040919,-0.004191,0.088923,-0.002979,0.023466,-0.013701,0.008769,0.002970,0.050793,0.049574,0.042552,1
50,0.224147,-2.651656,53.573863,0.181161,1.056194,0.211145,0.061918,0.016605,-0.030356,0.040919,-0.091484,0.011698,0.024391,0.007484,0.010337,0.005711,0.026541,0.049471,0.041361,1
51,0.860083,-1.949308,53.899868,0.187226,1.055450,-0.015685,0.060863,0.014445,0.016605,-0.030356,-0.004191,0.059787,-0.019248,0.008941,0.010944,0.003726,0.025933,0.049326,0.040430,0
52,0.665913,-1.426264,49.644855,0.186281,1.019012,-0.880709,0.062361,0.003092,0.014445,0.016605,0.040919,0.088923,-0.014890,-0.006367,0.006212,-0.000475,0.024908,0.051469,0.039905,1
53,0.738732,-0.993265,51.062147,0.186998,1.029017,0.254532,0.059498,-0.035620,0.003092,0.014445,-0.030356,-0.091484,-0.001232,0.002160,0.001461,0.002494,0.021738,0.048053,0.038436,0
